In [2]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os


In [3]:
#returns the value of location of square block and prediction
def detectpredict(frame, faceNet, maskNet):
    #left
    (h,w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (224,224),(104.0, 177.0, 123.0))      #pixels size 224x224 and mean subtracting
     #obtaining face detection                            
    faceNet.setInput(blob)
    detections = faceNet.forward()
    print(detections.shape)                                #shape of area detected
    
    # faces, their locations and prediction from facemask network
    faces=[]
    locs=[]
    preds=[]
    
    for i in range(0, detections.shape[2]):
        
        confidence = detections[0,0,i,2]
        #filter weak detections
        if confidence > 0.5:
            box= detections[0,0,i,3:7]* np.array([w,h,w,h])
            (startX, startY, endX, endY) = box.astype("int")
            (startX, startY)=(max(0,startX), max(0,startY))
            (endX, endY)= (min(w-1, endX), min(h-1, endY))
            
            #extract the roi, convert from bgr to rgb
            #set size 224,224
            face=frame[startX:endX, startY:endY]
            face=cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face=cv2.resize(face, (224,224))
            face=img_to_array(face)
            face=preprocess_input(face)
            
            faces.append(face)
            locs.append((startX,startY,endX,endY))
            
    if len(faces)>0:
        faces=np.array(faces, dtype="float32")
        preds=maskNet.predict(faces, batch_size=32)
        
    return (locs,preds)

prototxtPath = r"face_detector\deploy.prototxt"
weightsPath = r"face_detector\res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

maskNet = load_model("mask_detector.model")

print("starting video stream")
vs = VideoStream(src=0).start()         #src=0 for primary camera

while True:
    #frames from the video and resize to make max width 400
    frame = vs.read()
    frame = imutils.resize(frame, width=400)
    
    (locs, preds) = detectpredict(frame, faceNet, maskNet)
     
    for(box, pred) in zip(locs,preds):
        (startX,startY,endX,endY)=box
        (mask,without_mask)=pred
        
        label="mask present" if mask > without_mask else "No Mask"
        color= (0,255,0) if label=="mask present" else (0,0,225)  #BGR
        
        label = "{}:{:.2f}%".format(label, max(mask, without_mask)*100)         #label, %prediction using .format
        
        #display label and box
        cv2.putText(frame, label, (startX, startY-10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)   # cv2.putText(image, text, org, font, fontScale, color[], thickness[])
        cv2.rectangle(frame, (startX,startY),(endX,endY),color,2)       #above color coding
        
    #show frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) % 0xFF
    
    if key==ord("q"):    #end
        break

    
cv2.destroyAllWindows()
vs.stop()


starting video stream
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
